In [53]:
!pip3 install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [54]:
# Importing necessary libraries and packages

import requests  # Library for making HTTP requests
import torch  # PyTorch for deep learning
from PIL import Image  # Library for image processing
from transformers import *  # Transformers library for NLP tasks
from tqdm import tqdm  # Library for displaying progress bars
import numpy as np  # Library for numerical manipulation

device = "cuda" if torch.cuda.is_available() else "cpu"  # Checking for GPU availability and setting the device accordingly
device = torch.device('mps:0') if torch.backends.mps.is_available() else "cpu"

device = "cpu"
device

'cpu'

## Loading Encoders and Decoders

In [55]:
# The model used for encoding the image and extracting image features
# Available encoder models:
# encoder_model = "WinKawaks/vit-small-patch16-224"
# encoder_model = "google/vit-base-patch16-224"
encoder_model = "google/vit-base-patch16-224-in21k"
# encoder_model = "microsoft/swin-base-patch4-window7-224-in22k"

# The model used for decoding the image features and generating captions
# Available decoder models:
# decoder_model = "bert-base-uncased"
# decoder_model = "prajjwal1/bert-tiny"
# decoder_model = "gpt2"
decoder_model = "google-bert/bert-base-uncased"

## Load the pre-trained Encoder and Decoder models
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    encoder_model, decoder_model
).to(device)
# model.train()

loading configuration file config.json from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.42.3"
}

loading weights file model.safetensors from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/model.safetensors
All model checkpoint weights were used when initializing Vi

## Train a G-code tokenizer

In [56]:
import os
from tokenizers import CharBPETokenizer

# Initialize a tokenizer
tokenizer = CharBPETokenizer()

gcode_dir = "dataset/gcode"
gcode_files = [os.path.join(gcode_dir, f) for f in os.listdir(gcode_dir) if f.endswith('.txt')]

# Then train it!
tokenizer.train(gcode_files)

# Now, let's use it:
encoded = tokenizer.encode("G1 X1.0 Y1.0")

# And finally save it somewhere
tokenizer.save("./bpe-gcode_tokenizer.json")

encoded.tokens

['G1</w>', 'X1</w>', '.</w>', '0</w>', 'Y1</w>', '.</w>', '0</w>']

## Loading Tokenizers and Image Processors

In [57]:
## Initialize the Tokenizer

# The tokenizer is used to preprocess the text and convert it into numerical inputs for the model
# Available tokenizers:
# tokenizer = AutoTokenizer.from_pretrained("./bpe-gcode_tokenizer.json")
# tokenizer = GPT2TokenizerFast.from_pretrained(decoder_model)
# tokenizer = BertTokenizerFast.from_pretrained(decoder_model)
# Initialize the Tokenizer
tokenizer = PreTrainedTokenizerFast(tokenizer_file="./bpe-gcode_tokenizer.json")
tokenizer.add_special_tokens({'pad_token': '<pad>', 'eos_token': '</s>', 'bos_token': '<s>'})


## Initialize the Image Processor

# The image processor is used to preprocess the images and extract visual features
# Available image processors:
# - ViTImageProcessor (for "google/vit-base-patch16-224" and "microsoft/swin-base-patch4-window7-224-in22k" encoder models)
from transformers import AutoImageProcessor

# checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(encoder_model)
# image_processor = ViTImageProcessor.from_pretrained(encoder_model)

loading configuration file preprocessor_config.json from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/preprocessor_config.json
loading configuration file config.json from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.42.3"
}

Fast image processor class <class 'tran

# Importing and Loading the Dataset

In [58]:
# from datasets import load_dataset

# max_length = 32  # Maximum length of the captions in tokens
# coco_dataset_ratio = 50  # 50% of the COCO2014 dataset

# # Load the COCO2014 dataset for training, validation, and testing splits
# train_ds = load_dataset("HuggingFaceM4/COCO", trust_remote_code=True, split=f"train[:{coco_dataset_ratio}%]")
# valid_ds = load_dataset("HuggingFaceM4/COCO", trust_remote_code=True, split=f"validation[:{coco_dataset_ratio}%]")
# test_ds = load_dataset("HuggingFaceM4/COCO", trust_remote_code=True, split="test")

# # Get the number of examples in each split
# train_len = len(train_ds)
# valid_len = len(valid_ds)
# test_len = len(test_ds)

# train_len, valid_len, test_len  # Display the number of examples in each split

In [59]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from transformers import PreTrainedTokenizerFast

# Load the custom tokenizer
tokenizer = PreTrainedTokenizerFast(tokenizer_file="./bpe-gcode_tokenizer.json")
tokenizer.add_special_tokens({'pad_token': '<pad>', 'eos_token': '</s>', 'bos_token': '<s>'})

# Dataset class to handle image and G-code pairs
class ImageGCodeDataset(Dataset):
    def __init__(self, image_dir, gcode_dir, transform=None, tokenizer=None):
        self.image_dir = image_dir
        self.gcode_dir = gcode_dir
        self.transform = transform
        self.tokenizer = tokenizer
        self.image_files = sorted(os.listdir(image_dir))
        self.gcode_files = sorted(os.listdir(gcode_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        gcode_path = os.path.join(self.gcode_dir, self.gcode_files[idx])
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        with open(gcode_path, 'r', encoding='utf-8', errors='ignore') as f:
            gcode = f.read()

        # if self.tokenizer:
        #     gcode = self.tokenizer(gcode, return_tensors='pt', padding='max_length', truncation=True, max_length=512)

        # return {"pixel_values": image, "labels": gcode['input_ids'].squeeze()}
        return {"pixel_values": image, "labels": gcode}

# Function to load the dataset
def load_dataset(image_dir, gcode_dir, tokenizer):
    # Define the image transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize the images to 224x224 pixels
        transforms.ToTensor(),          # Convert the images to PyTorch tensors
    ])
    
    # Create the dataset object
    dataset = ImageGCodeDataset(image_dir, gcode_dir, transform, tokenizer)
    return dataset

In [60]:
dataset = load_dataset('./dataset/images', './dataset/gcode', tokenizer)

In [61]:
dataset

In [62]:
dataset = list(dataset)

train_ds = dataset[0:int(len(dataset)*0.5)]
valid_ds = dataset[int(len(dataset)*0.5):int(len(dataset)*0.75)]
test_ds = dataset[int(len(dataset)*0.75):]

In [63]:
train_ds[0]

{'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 'labels': 'G21 ; Set units to millimeters\nG90 ; Absolute positioning\nG1 X4.329 Y3.405 F1200\nG1 X4.214 Y4.499 F1200\nG1 X3.114 Y4.479 F1200\nG1 X2.999 Y3.385 F1200'}

In [64]:
max_length = 32

# def preprocess(items):
#     # Preprocess the image
#     pixel_values = image_processor(items["pixel_values"], return_tensors="pt", do_rescale=False).pixel_values.squeeze(0).to(device)


#     # Tokenize the captions with truncation and padding
#     targets = tokenizer(items["labels"], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt").to(device)
#     # targets = tokenizer(items["labels"], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt").input_ids.squeeze(0).to(device)

#     return {'pixel_values': pixel_values, 'labels': targets["input_ids"]}

image_processor = AutoImageProcessor.from_pretrained(encoder_model)

def preprocess(items):
    
    # Preprocess the image
    pixel_values = image_processor(items["pixel_values"], return_tensors="pt", do_rescale=False).pixel_values.squeeze(0).float().to(device)
    
    # Tokenize the captions with truncation and padding
    targets = tokenizer(items["labels"], max_length=max_length, padding="max_length", truncation=True, return_tensors="pt").input_ids.to(device)
    print(targets)
    return {'pixel_values': pixel_values, 'labels': targets}

print(train_ds[0])

# Apply the preprocess function to transform the datasets during training
train_dataset = list(map(preprocess, train_ds))
valid_dataset = list(map(preprocess, valid_ds))
test_dataset = list(map(preprocess, test_ds))

print(train_dataset[0])

loading configuration file preprocessor_config.json from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/preprocessor_config.json
loading configuration file config.json from cache at /Users/matth/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/b4569560a39a0f1af58e3ddaf17facf20ab919b0/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.42.3"
}

Fast image processor class <class 'tran

{'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         ...,
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.],
         [1., 1., 1.,  ..., 1., 1., 1.]]]), 'labels': 'G21 ; Set units to millimeters\nG90 ; Absolute positioning\nG1 X4.329 Y3.405 F1200\nG1 X4.214 Y4.499 F1200\nG1 X3.114 Y4.479 F1200\nG1 X2.999 Y3.385 F1200'}
tensor([[ 68,  51,  91,  98,

In [65]:
valid_dataset[0]

{'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 'labels': tensor([[ 68,  51,  91,  98,  87,  95,  69,  51,  99, 100,  56,  59,  47, 535,
           58,  47, 539,  57,  56,  59,  47, 488,  60,  47, 363,  57,  56,  61,
    

# Metrics Computation

In [66]:
def compute_metrics(eval_pred):
    preds = eval_pred.predictions
    labels = eval_pred.label_ids

    # Decode the predictions and labels
    pred_str = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels_str = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute the Rouge score
    rouge_result = rouge.compute(predictions=pred_str, references=labels_str)
    rouge_result = {k: round(v * 100, 4) for k, v in rouge_result.items()}  # Multiply by 100 to get the same scale as the Rouge score

    # Compute the Bleu score
    bleu_result = bleu.compute(predictions=pred_str, references=labels_str)

    # Get the length of the generated captions
    generation_length = bleu_result["translation_length"]

    return {
        **rouge_result,
        "bleu": round(bleu_result["bleu"] * 100, 4),
        "gen_len": bleu_result["translation_length"] / len(preds)
    }

# Training Parameters

In [67]:
num_epochs = 1  # Number of epochs
batch_size = 1  # Batch size

# Set the number of training epochs and the batch size. Adjust these values 
# according to your specific requirements.

# Dataset Example Shapes

In [68]:
# Iterate over the training dataset and print the shapes of 
# labels and pixel values for an example.
for item in valid_dataset:
    print(item["labels"].shape)
    print(item["pixel_values"].shape)
    # break

torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3, 224, 224])
torch.Size([1, 32])
torch.Size([3,

In [69]:
test_dataset[0]

{'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]],
 
         [[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]]),
 'labels': tensor([[ 68,  51,  91,  98,  87,  95,  69,  51,  99, 100,  56, 103,  47, 121,
           40,  58,  47, 760,  57,  56, 103,  47, 140,  42, 102,  47, 815,  57,
    

# Batch Collation Function

In [70]:
# def collate_fn(batch):
#     print(batch.shape)
#     print(torch.stack([x['pixel_values'] for x in batch]).shape)
#     return {
#         'pixel_values': torch.stack(torch.tensor([x['pixel_values'] for x in batch])),
#         'labels': torch.stack(torch.tensor([x['labels'] for x in batch]))
#     }

# This function takes a batch of preprocessed examples and stacks the pixel 
# values and labels into tensors. It will be used by the data loader to collate the samples into batches.

In [71]:
# Batch Collation Function
def collate_fn(batch):
    # Extract pixel values and labels
    pixel_values = torch.stack([item['pixel_values'] for item in batch]).to(device)
    labels = torch.stack([item['labels'] for item in batch]).to(device)

    return {
        'pixel_values': pixel_values,
        'labels': labels
    }

# Training Arguments

In [72]:
from transformers import Seq2SeqTrainingArguments

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,             # Use generate to calculate the loss
    num_train_epochs=num_epochs,            # Number of training epochs
    eval_strategy="steps",            # Evaluate after each eval_steps
    eval_steps=2000,                        # Evaluate after each 2000 steps
    logging_steps=2000,                     # Log after each 2000 steps
    save_steps=2000,                        # Save after each 2000 steps
    per_device_train_batch_size=batch_size, # Batch size for training
    per_device_eval_batch_size=batch_size,  # Batch size for evaluation
    output_dir="vit-swin-base-224-gpt2-gcode-gen",  # Output directory for saving checkpoints and logs
    # push_to_hub=True # Whether you want to push the model to the hub
    # Check this guide for more details: https://huggingface.co/transformers/model_sharing.html
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [73]:
num_channels, height, width = train_dataset[40]['pixel_values'].shape
num_channels, height, width

(3, 224, 224)

In [74]:
height, width = train_dataset[40]['labels'].shape
height, width

(1, 32)

In [75]:
train_dataset[40]['pixel_values'][0]

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [76]:
from transformers import Trainer

trainer = Trainer(model=model, 
                  args=training_args, 
                  data_collator=collate_fn,
                  train_dataset=train_dataset,
                  eval_dataset=valid_dataset,
                  tokenizer=tokenizer
                  )

# DataLoader Functions

In [77]:
from torch.utils.data import DataLoader

def get_eval_loader(eval_dataset=None):
    return DataLoader(valid_dataset, collate_fn=collate_fn, batch_size=batch_size)

def get_test_loader(eval_dataset=None):
    return DataLoader(test_dataset, collate_fn=collate_fn, batch_size=batch_size)

# Override the `get_train_dataloader`, `get_eval_dataloader`, and `get_test_dataloader` methods of the trainer
# so that we can properly load the data

trainer.get_train_dataloader = lambda: DataLoader(train_dataset, collate_fn=collate_fn, batch_size=batch_size)
trainer.get_eval_dataloader = get_eval_loader
trainer.get_test_dataloader = get_test_loader

# These functions define the data loaders for training, evaluation, and testing. 
# We override the default methods in the trainer to use our custom data loaders 
# that properly collate the batches using the `collate_fn` function.

# Train the model

In [78]:
trainer.train()

# Evaluate the model on the test dataset
# trainer.evaluate(test_dataset)

# If you set the push_to_hub parameter in the TrainingArguments,
# complete the push to the model hub using the following code
# trainer.push_to_hub()

***** Running training *****
  Num examples = 50
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 50
  Number of trainable parameters = 224,270,394


  0%|          | 0/50 [00:00<?, ?it/s]

RuntimeError: slow_conv2d_forward_mps: input(device='cpu') and weight(device=mps:0')  must be on the same device

In [ ]:

# from transformers import AutoProcessor, VisionEncoderDecoderModel
# import requests
# from PIL import Image
# import torch

# processor = AutoProcessor.from_pretrained("microsoft/trocr-base-handwritten")
# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# # load image from the IAM dataset
# url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

# # training
# model.config.decoder_start_token_id = processor.tokenizer.eos_token_id
# model.config.pad_token_id = processor.tokenizer.pad_token_id
# model.config.vocab_size = model.config.decoder.vocab_size

# pixel_values = processor(image, return_tensors="pt").pixel_values
# text = "hello world"
# labels = processor.tokenizer(text, return_tensors="pt").input_ids
# outputs = model(pixel_values=pixel_values, labels=labels)
# loss = outputs.loss

# # inference (generation)
# generated_ids = model.generate(pixel_values)
# generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
loss, generated_text